# Loan Default Prediction - Part 01 - Data Preprocessing

This is an exploratory project for me to apply what I have learned in the Data Science and Machine Learning courses that I took this year. The data is from a Kaggle competition [Loan Default Prediction](https://www.kaggle.com/c/loan-default-prediction). 

This is Part 01 of the project: Data Preprocessing.

## Data: A First Glance

The data has one __id__ column, 769 feature columns named __f1__ ... __f778__, and one target column __loss__ indicating the percentage of loan unpaid for. There are 105,471 rows. I am using the training data only because __loss__ is not included in the testing data provided on the Kaggle page. 

The contest organizer is cautious about data privacy and anonymity because the data was collected from real people. As a result, there is no description or explanation of what each feature column means. It's a double-edged sword -- I won't be able to take advantage of the nature of the data, but I also won't be liberally applying heuristics that I _think_ will be helpful but actually have limited use.

In [1]:
# Import libraries

import numpy as np
import pandas as pd

In [2]:
# Load the training data from the CSV file

file = 'train_v2.csv'
df_raw = pd.read_csv(file, na_values='NA')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (135,204,274,417) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Display basic information

df_raw.info()
print(df_raw.describe(include='all'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105471 entries, 0 to 105470
Columns: 771 entries, id to loss
dtypes: float64(653), int64(99), object(19)
memory usage: 620.4+ MB
                   id             f1             f2             f3  \
count   105471.000000  105471.000000  105471.000000  105471.000000   
unique            NaN            NaN            NaN            NaN   
top               NaN            NaN            NaN            NaN   
freq              NaN            NaN            NaN            NaN   
mean     52736.000000     134.603171       8.246883       0.499066   
std      30446.999458      14.725467       1.691535       0.288752   
min          1.000000     103.000000       1.000000       0.000006   
25%      26368.500000     124.000000       8.000000       0.248950   
50%      52736.000000     129.000000       9.000000       0.498267   
75%      79103.500000     148.000000       9.000000       0.749494   
max     105471.000000     176.000000      11.000000

When loading the CSV file, I got some warnings about some columns having mixed types. As they may be more tricky to process, I decide to drop them for now. After dropping them, the number of columns reduces from 771 to 751.

In [4]:
cols_to_drop = []

# Identifying non-numerical features    
for c in df_raw.columns:
    if (df_raw[c].dtypes == object):
        cols_to_drop.append(c)
            
print("Columns to drop: ", cols_to_drop)

df_raw = df_raw.drop(cols_to_drop, axis=1)
df_raw.info()

Columns to drop:  ['f137', 'f138', 'f206', 'f207', 'f276', 'f277', 'f338', 'f390', 'f391', 'f419', 'f420', 'f469', 'f472', 'f534', 'f537', 'f626', 'f627', 'f695', 'f698']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105471 entries, 0 to 105470
Columns: 752 entries, id to loss
dtypes: float64(653), int64(99)
memory usage: 605.1 MB


## Imputing Missing Data

Some columns contain missing values. I iterate over the columns to find which columns have missing values and how many entries they are missing.

In [5]:
# Look for columns with missing values

cols_with_missing_vals = dict()
for c in df_raw.columns:
    if df_raw[c].isnull().sum() > 0:
        cols_with_missing_vals[c] = df_raw[c].isnull().sum()

import pprint
pp = pprint.PrettyPrinter(indent=4)
print("Columns with missing values:")
pp.pprint(cols_with_missing_vals)  

Columns with missing values:
{   'f100': 1704,
    'f101': 1704,
    'f102': 335,
    'f103': 335,
    'f104': 182,
    'f105': 182,
    'f106': 182,
    'f107': 182,
    'f108': 182,
    'f109': 182,
    'f110': 1491,
    'f111': 1491,
    'f112': 335,
    'f113': 335,
    'f114': 182,
    'f115': 182,
    'f116': 182,
    'f117': 182,
    'f118': 182,
    'f119': 182,
    'f120': 1121,
    'f121': 1121,
    'f122': 335,
    'f123': 335,
    'f124': 182,
    'f125': 182,
    'f126': 182,
    'f127': 182,
    'f128': 182,
    'f129': 182,
    'f130': 808,
    'f131': 808,
    'f132': 335,
    'f133': 335,
    'f134': 182,
    'f135': 182,
    'f136': 182,
    'f139': 182,
    'f14': 100,
    'f140': 682,
    'f141': 682,
    'f142': 2561,
    'f143': 2561,
    'f144': 1291,
    'f145': 1291,
    'f146': 1291,
    'f147': 1291,
    'f148': 1291,
    'f149': 2859,
    'f15': 48,
    'f150': 2859,
    'f151': 2561,
    'f152': 2561,
    'f153': 1291,
    'f154': 1291,
    'f155': 1291,
  

It turns out there are many columns with missing values. I use [sklearn.preprocessing.Imputer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html) to fill the missing values. I choose the _most frequent_ imputation stratety because I believe some of the columns may be categorical features, and _mean_ or _median_ wouldn't make much sense.

I apply the Imputer using the [sklearn-pandas](https://github.com/scikit-learn-contrib/sklearn-pandas) DataFrameMapper framework so that the transformed data stays as a DataFrame. After imputing the data, there are no more missing values.

In [6]:
from sklearn.preprocessing import Imputer
from sklearn_pandas import gen_features
from sklearn_pandas import DataFrameMapper

cols_to_impute = []
for key, value in cols_with_missing_vals.items():
    cols_to_impute.append([key])

feature_def = gen_features(
    columns=cols_to_impute,
    classes=[{'class': Imputer, 'strategy': 'most_frequent'}]
)

mapper = DataFrameMapper(feature_def, default=None, input_df=True, df_out=True)
df_imputed = mapper.fit_transform(df_raw)
df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105471 entries, 0 to 105470
Columns: 752 entries, f7 to loss
dtypes: float64(752)
memory usage: 605.1 MB


In [7]:
cols_with_missing_vals = dict()
for c in df_imputed.columns:
    if df_imputed[c].isnull().sum() > 0:
        cols_with_missing_vals[c] = df_imputed[c].isnull().sum()
        
pp = pprint.PrettyPrinter(indent=4)
print("Columns with missing values:")
pp.pprint(cols_with_missing_vals)  

Columns with missing values:
{}


## One-Hot Encoding for Categorical Features

As the values of categorical features have no physical meaning, they are usually transformed into one-hot encoding before being used to train any models. For this particular dataset, a tricky part is that we don't know which features are categorical. I can only assume that a column is likely to be categorical if (1) it is entirely composed of integer values and (2) the number of unique values is small (i.e. under a certain threshold value). I apply this rule to the columns to determine which columns are categorical, and then I use [sklearn.preprocessing.OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) to transform these columns to one-hot encoding. 

Using the DataFrameMapper framework, the new one-hot-encoded columns are systematically named. For example, column __f1__ has 74 unique values. After the tranformation, __f1__ is removed, and 74 new columns __f1_0__ ... __f1_73__ are added.

In addition, if any column contains only one value, it has no information, and we can safely drop it.

In [8]:
# Identify categorical features and single-valued features

# Define categorical_threshold
# If a column's number of unique values is less than this value, consider it categorical
categorical_threshold = 100

# Identifying categorical features
# Using df_raw because df_imputed is all float
cols_single_val = []
cols_categorical = []
cols_continuous = [] # non-categorical
for c in df_raw.columns:
    if (c == 'id' or c == 'loss'):
        continue
    if (df_raw[c].dtypes == 'int64' and len(df_raw.loc[:,c].unique()) <= categorical_threshold):
        print("Column \"" + c + "\" has unique values: ")
        print(df_raw.loc[:,c].unique())
        if (len(df_raw.loc[:,c].unique()) == 1):
            cols_single_val.append(c)
        else:
            cols_categorical.append([c])
    else:
        cols_continuous.append([c])

# After this check, we no longer need df_raw
del df_raw
            
print("Categorical columns: ", cols_categorical)
print("Single-valued columns: ", cols_single_val)

Column "f1" has unique values: 
[126 121 134 109 128 127 115 120 130 119 116 123 125 114 129 117 124 122
 139 144 142 137 154 150 147 156 157 146 153 135 138 118 113 133 110 111
 136 132 108 106 152 145 149 155 163 166 165 160 148 159 151 162 158 131
 143 161 164 140 141 173 167 112 168 107 170 105 171 103 104 169 174 176
 172 175]
Column "f2" has unique values: 
[10  9  8  6  7  4  2  3  1 11]
Column "f4" has unique values: 
[1100 2900 1300 1800 2200 3800 2600 1600 3500 1900 2700 4800 4200 3600
 3700 1500 2400 4300 5400 2000 3100 5500 5000 4400 4700 2300 5100 5300
 1200 3900 6200 5900 5700 4100 4000 4500 1700 2500 6600 3300 2100 3000
 5800 3200 2800 4600 5600 7200 3400 1400 5200 6100 6800 7000 6500 6300
 6400 7300 6700 7400 7900]
Column "f5" has unique values: 
[ 3  4 16 10  2  7 15  1 17 13]
Column "f13" has unique values: 
[ 7  6 14 13 15  8 10 11  4 16 12  5  9  3 17 18 20 19 25 29 28 27 23 26
 21 24 22 32  2 40 30]
Column "f33" has unique values: 
[0]
Column "f34" has unique value

In [9]:
# Drop one-value columns

df_imputed = df_imputed.drop(cols_single_val, axis=1)
df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105471 entries, 0 to 105470
Columns: 742 entries, f7 to loss
dtypes: float64(742)
memory usage: 597.1 MB


In [10]:
# Replace categorical features with one-hot encoding

from sklearn.preprocessing import OneHotEncoder

feature_def = gen_features(
    columns=cols_categorical,
    classes=[OneHotEncoder] 
)

mapper = DataFrameMapper(feature_def, default=None, input_df=True, df_out=True)
df_imputed = mapper.fit_transform(df_imputed)
df_imputed.info()
print(df_imputed.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105471 entries, 0 to 105470
Columns: 1790 entries, f1_0 to loss
dtypes: float64(1790)
memory usage: 1.4 GB
                f1_0           f1_1           f1_2           f1_3  \
count  105471.000000  105471.000000  105471.000000  105471.000000   
mean        0.000284       0.000076       0.000294       0.000218   
std         0.016863       0.008709       0.017142       0.014766   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.000000       0.000000       0.000000   
50%         0.000000       0.000000       0.000000       0.000000   
75%         0.000000       0.000000       0.000000       0.000000   
max         1.000000       1.000000       1.000000       1.000000   

                f1_4           f1_5           f1_6           f1_7  \
count  105471.000000  105471.000000  105471.000000  105471.000000   
mean        0.001185       0.000578       0.002266       0.002579   
std         0.

In [11]:
# Store the preprocessded DataFrame into a new CSV

df_imputed.to_csv('train_v2_preprocessed.csv')

## Normalizing Continuous Features

Now that I've preprocessed categorical features, it's time to look at the continuous features (non-categorical). From the printed column information, we can see that the values in these columns are distributed differently. For example, column __f7__ has a range of [1, 9968], while column __f766__ has a range of (-0.5, 0]. The value range can greatly impact the performance of some machine learning models, so I will prepare a normalized version of the DataFrame. I use [sklearn.preprocessing.StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to scale these features to have mean 0 and standard deviation 1.

In [12]:
from sklearn.preprocessing import StandardScaler

feature_def = gen_features(
    columns=cols_continuous,
    classes=[StandardScaler]
)

mapper = DataFrameMapper(feature_def, default=None, input_df=True, df_out=True)
df_imputed_normalized = mapper.fit_transform(df_imputed)
df_imputed_normalized.info()
print(df_imputed_normalized.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105471 entries, 0 to 105470
Columns: 1790 entries, f3 to loss
dtypes: float64(1790)
memory usage: 1.4 GB
                 f3            f6            f7            f8            f9  \
count  1.054710e+05  1.054710e+05  1.054710e+05  1.054710e+05  1.054710e+05   
mean   5.622168e-17 -2.554207e-15  5.490115e-17  7.954330e-17  1.151258e-15   
std    1.000005e+00  1.000005e+00  1.000005e+00  1.000005e+00  1.000005e+00   
min   -1.728340e+00 -1.345229e+00 -1.165221e+00 -1.075084e+00 -2.006220e+00   
25%   -8.662007e-01 -1.029760e+00 -9.206516e-01 -7.467748e-01 -7.425324e-01   
50%   -2.767762e-03  7.998521e-01 -2.685968e-01 -2.872299e-01 -4.408965e-01   
75%    8.672785e-01  9.008977e-01  6.692472e-01  4.305880e-01  1.050646e+00   
max    1.734808e+00  1.137185e+00  2.747500e+00  4.024096e+00  2.777276e+00   

                f10           f14           f15           f16           f17  \
count  1.054710e+05  1.054710e+05  1.054710e+05  1.054

In [13]:
# Store the normalized DataFrame into a new CSV

df_imputed_normalized.to_csv('train_v2_preprocessed_normalized.csv')

## Feature Expansion

The continuous features can also be expanded polynomially, for example, adding x^2 or x^3 terms, to account for some of the nonlinear dependency. I use [sklearn.preprocessing.PolynomialFeatures](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) to achieve this.

In [14]:
# Expand continuous features to be polynomial (degree 2)

from sklearn.preprocessing import PolynomialFeatures

feature_def = gen_features(
    columns=cols_continuous,
    classes=[{'class': PolynomialFeatures, 'degree': 2}]
)

mapper = DataFrameMapper(feature_def, default=None, input_df=True, df_out=True)
df_poly2 = mapper.fit_transform(df_imputed_normalized)
df_poly2.info()
print(df_poly2.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105471 entries, 0 to 105470
Columns: 3200 entries, f3_1 to loss
dtypes: float64(3200)
memory usage: 2.5 GB
           f3_1         f3_x0       f3_x0^2      f6_1         f6_x0  \
count  105471.0  1.054710e+05  1.054710e+05  105471.0  1.054710e+05   
mean        1.0  5.622168e-17  1.000000e+00       1.0 -2.554207e-15   
std         0.0  1.000005e+00  8.938727e-01       0.0  1.000005e+00   
min         1.0 -1.728340e+00  1.335714e-09       1.0 -1.345229e+00   
25%         1.0 -8.662007e-01  1.872665e-01       1.0 -1.029760e+00   
50%         1.0 -2.767762e-03  7.511669e-01       1.0  7.998521e-01   
75%         1.0  8.672785e-01  1.689483e+00       1.0  9.008977e-01   
max         1.0  1.734808e+00  3.009558e+00       1.0  1.137185e+00   

             f6_x0^2      f7_1         f7_x0       f7_x0^2      f8_1  \
count  105471.000000  105471.0  1.054710e+05  1.054710e+05  105471.0   
mean        1.000000       1.0  5.490115e-17  1.000000e+00 

In [15]:
# Store the expanded DataFrame into a new CSV

df_poly2.to_csv('train_v2_preprocessed_normalized_poly2.csv')

In [16]:
# Expand continuous features to be polynomial (degree 3)

feature_def = gen_features(
    columns=cols_continuous,
    classes=[{'class': PolynomialFeatures, 'degree': 3}]
)

mapper = DataFrameMapper(feature_def, default=None, input_df=True, df_out=True)
df_poly3 = mapper.fit_transform(df_imputed_normalized)
df_poly3.info()
print(df_poly3.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105471 entries, 0 to 105470
Columns: 3905 entries, f3_1 to loss
dtypes: float64(3905)
memory usage: 3.1 GB
           f3_1         f3_x0       f3_x0^2       f3_x0^3      f6_1  \
count  105471.0  1.054710e+05  1.054710e+05  1.054710e+05  105471.0   
mean        1.0  5.622168e-17  1.000000e+00  3.568024e-03       1.0   
std         0.0  1.000005e+00  8.938727e-01  1.962592e+00       0.0   
min         1.0 -1.728340e+00  1.335714e-09 -5.162823e+00       1.0   
25%         1.0 -8.662007e-01  1.872665e-01 -6.499135e-01       1.0   
50%         1.0 -2.767762e-03  7.511669e-01 -2.120246e-08       1.0   
75%         1.0  8.672785e-01  1.689483e+00  6.523427e-01       1.0   
max         1.0  1.734808e+00  3.009558e+00  5.221005e+00       1.0   

              f6_x0        f6_x0^2        f6_x0^3      f7_1         f7_x0  \
count  1.054710e+05  105471.000000  105471.000000  105471.0  1.054710e+05   
mean  -2.554207e-15       1.000000      -0.211704

In [17]:
# Store the expanded DataFrame into a new CSV

df_poly3.to_csv('train_v2_preprocessed_normalized_poly3.csv')